# Quickstart: Vector search using Azure OpenAI Embeddings and Elasticsearch

This tutorial demonstrates how to use the [Azure OpenAI API](https://azure.microsoft.com/en-in/products/ai-services/openai-service) to create [embeddings](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/embeddings?tabs=console) and store them in Elasticsearch. Elasticsearch will enable us to perform vector search (Knn) to find similar documents.

## setup

* Elastic Credentials - Create [Cloud deployment](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) to get all Elastic credentials (`ELASTIC_CLOUD_ID`, `ELASTIC_API_KEY`).

* `AZURE_OPENAI_API_KEY` - To use the Azure OpenAI API, you need an API key. [Follow](https://learn.microsoft.com/en-us/azure/ai-services/openai/quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python#retrieve-key-and-endpoint) to create a key.
* `AZURE_OPENAI_ENDPOINT` - Endpoint for your Azure OpenAI Resource.
* `AZURE_DEPLOYMENT_ID` - The deployment name you chose when you deployed the model.
* `AZURE_OPENAI_API_VERSION` - The API version to use for this operation. This follows the YYYY-MM-DD format.

## Install packages

In [ ]:
pip install -q -U openai elasticsearch

## Import packages and credentials

In [ ]:
from openai import AzureOpenAI
from elasticsearch import Elasticsearch, helpers
from getpass import getpass
import os

ELASTIC_API_KEY = getpass("Elastic API Key :")
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID :")

AZURE_OPENAI_API_KEY = getpass("Azure OpenAI API Key :")
AZURE_OPENAI_ENDPOINT = getpass("Azure OpenAI Endpoint :")
AZURE_DEPLOYMENT_ID = getpass("Azure Deployment ID :")
AZURE_OPENAI_API_VERSION = getpass("Azure OpenAI API Version :")

elastic_index_name = "azure-openai-vector-search-demo"

## Embedding generation



In [ ]:
def generate_embeddings(text):
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
    )

    response = client.embeddings.create(
        input=text,
        model=AZURE_DEPLOYMENT_ID,
    )

    return response.data[0].embedding


sample_text = "India generally experiences a hot summer from March to June, with temperatures often exceeding 40°C in central and northern regions. Monsoon season, from June to September, brings heavy rainfall, especially in the western coast and northeastern areas. Post-monsoon months, October and November, mark a transition with decreasing rainfall. Winter, from December to February, varies in temperature across the country, with colder conditions in the north and milder weather in the south. India's diverse climate is influenced by its geographical features, resulting in regional"
embeddings = generate_embeddings(sample_text)

## Connecting Elasticsearch

In [ ]:
es = Elasticsearch(cloud_id=ELASTIC_CLOUD_ID, api_key=ELASTIC_API_KEY)

## Index document with Elasticsearch

In [ ]:
doc = {"text": sample_text, "text_embedding": embeddings}

resp = es.index(index=elastic_index_name, document=doc)

print(resp)

## Searching for document with Elasticsearch

In [7]:
q = "How's weather in India?"

embeddings = generate_embeddings(q)

resp = es.search(
    index=elastic_index_name,
    knn={
        "field": "text_embedding",
        "query_vector": embeddings,
        "k": 10,
        "num_candidates": 100,
    },
)


for result in resp["hits"]["hits"]:
    pretty_output = f"\n\nID: {result['_id']}\n\nText: {result['_source']['text']}"
    print(pretty_output)



ID: SxtQyY4BMvvuJ06pSACG

Text: India generally experiences a hot summer from March to June, with temperatures often exceeding 40°C in central and northern regions. Monsoon season, from June to September, brings heavy rainfall, especially in the western coast and northeastern areas. Post-monsoon months, October and November, mark a transition with decreasing rainfall. Winter, from December to February, varies in temperature across the country, with colder conditions in the north and milder weather in the south. India's diverse climate is influenced by its geographical features, resulting in regional
